# **Simple Tensorflow Regression**

This notebook is a copy of Mario Filho's tuturial about using tensorflow framework to train a neural network for regression. The original video is [aqui](https://www.youtube.com/watch?v=s0s6Q1GLJGo).

Os dados utilizados foram retirado da competição House Prices do site Kaggle e pode ser encontrados [aqui](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data).

In [ ]:
!pip install category_encoders

In [63]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import OneHotEncoder
from category_encoders import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MaxAbsScaler

import tensorflow as tf
from tensorflow import keras as tfk

%matplotlib inline
pd.set_option("display.max_columns", 81)
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
# Carrega os dados e separa o dataset X e y
data = pd.read_csv("data/train.csv")
X = data.drop(["Id","SalePrice"], axis=1)
y = data['SalePrice']

In [5]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [47]:
Xtr, Xval, ytr, yval = train_test_split(X, y, train_size=0.5, random_state=0)
Xtr.shape, Xval.shape, ytr.shape, yval.shape

((730, 79), (730, 79), (730,), (730,))

In [10]:
Xtr.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
733,20,RL,80.0,10000,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,Sawyer,Feedr,Norm,1Fam,1Story,5,6,1961,1983,Hip,CompShg,HdBoard,HdBoard,None,0.0,TA,TA,CBlock,TA,TA,No,BLQ,594,Unf,0,270,864,GasA,Ex,Y,SBrkr,1144,0,0,1144,1,0,1,0,3,1,TA,6,Typ,1,TA,Attchd,1961.0,RFn,1,264,TA,TA,Y,165,0,0,0,0,0,NaN,GdWo,Shed,400,3,2009,WD,Normal
382,60,RL,79.0,9245,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2006,2006,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,Av,Unf,0,Unf,0,939,939,GasA,Ex,Y,SBrkr,939,858,0,1797,0,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2006.0,RFn,2,639,TA,TA,Y,144,53,0,0,0,0,NaN,NaN,NaN,0,4,2007,WD,Normal
1232,90,RL,70.0,9842,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,NAmes,Norm,Norm,Duplex,1Story,4,5,1962,1962,Gable,CompShg,HdBoard,HdBoard,None,0.0,TA,TA,Slab,NaN,NaN,NaN,NaN,0,NaN,0,0,0,GasA,TA,Y,SBrkr,1224,0,0,1224,0,0,2,0,2,2,TA,6,Typ,0,NaN,CarPort,1962.0,Unf,2,462,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,3,2007,WD,Normal
1354,60,RL,NaN,10316,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,No,GLQ,735,Unf,0,257,992,GasA,Ex,Y,SBrkr,992,873,0,1865,1,0,2,1,3,1,Gd,7,Typ,1,TA,Attchd,2000.0,RFn,3,839,TA,TA,Y,0,184,0,0,0,0,NaN,NaN,NaN,0,6,2008,WD,Normal
331,20,RL,70.0,8176,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,6,1958,1992,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,846,Unf,0,210,1056,GasA,Fa,Y,SBrkr,1056,0,0,1056,1,0,1,0,3,1,TA,6,Typ,0,NaN,Attchd,1958.0,RFn,1,308,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal


**Encodding de variáveis categóricas**

As redes neurais não trabalham com dados categóricos, assim, temos que aplicar alguma técnicas para tansformar os dados das coluna categóricas em numéricas. Uma forma simples é o **OneHot Encodding**. Esta técnica usará os valores para montar uma lista de coluna correspndendo a presença ou não do valor categórico. Esse tipo de transformação vai adicionar novas colunas no dataset, uma coluna para cada valor da categoria. Outro fator importante que é o transformador utilizado poderá ser reutilizado para reverter a transformação quando necessário.

In [48]:
# list de colunas categoricas do dataset
cat_cols = X.select_dtypes(include=['object']).copy().columns.tolist()

In [49]:
# aplicando o OneHotEncoding
cat_encoder = OneHotEncoder(cols=cat_cols, drop_invariant=True, handle_missing=0)
Xtr = cat_encoder.fit_transform(Xtr)
Xval = cat_encoder.transform(Xval)

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [50]:
Xtr.head()

,MSSubClass,MSZoning_1,MSZoning_2,MSZoning_3,MSZoning_4,MSZoning_5,LotFrontage,LotArea,Street_1,Street_2,Alley_1,Alley_2,Alley_3,LotShape_1,LotShape_2,LotShape_3,LotShape_4,LandContour_1,LandContour_2,LandContour_3,LandContour_4,LotConfig_1,LotConfig_2,LotConfig_3,LotConfig_4,LandSlope_1,LandSlope_2,LandSlope_3,Neighborhood_1,Neighborhood_2,Neighborhood_3,Neighborhood_4,Neighborhood_5,Neighborhood_6,Neighborhood_7,Neighborhood_8,Neighborhood_9,Neighborhood_10,Neighborhood_11,Neighborhood_12,...,GarageCond_4,GarageCond_5,PavedDrive_1,PavedDrive_2,PavedDrive_3,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC_1,PoolQC_2,PoolQC_3,Fence_1,Fence_2,Fence_3,Fence_4,Fence_5,MiscFeature_1,MiscFeature_2,MiscFeature_3,MiscFeature_4,MiscVal,MoSold,YrSold,SaleType_1,SaleType_2,SaleType_3,SaleType_4,SaleType_5,SaleType_6,SaleType_7,SaleType_8,SaleCondition_1,SaleCondition_2,SaleCondition_3,SaleCondition_4,SaleCondition_5,SaleCondition_6
733,20,1,0,0,0,0,80.0,10000,1,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,165,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,400,3,2009,1,0,0,0,0,0,0,0,1,0,0,0,0,0
382,60,1,0,0,0,0,79.0,9245,1,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,144,53,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,4,2007,1,0,0,0,0,0,0,0,1,0,0,0,0,0
1232,90,1,0,0,0,0,70.0,9842,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,3,2007,1,0,0,0,0,0,0,0,1,0,0,0,0,0
1354,60,1,0,0,0,0,NaN,10316,1,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,184,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,6,2008,1,0,0,0,0,0,0,0,1,0,0,0,0,0
331,20,1,0,0,0,0,70.0,8176,1,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,8,2007,1,0,0,0,0,0,0,0,1,0,0,0,0,0


**Imputter**

Precisamos agora tratar os campos que estão sem valores. Faremos isso utiizando um Imputter.

In [51]:
# imputando valores ausente com a média dos valores da coluna
imputer = SimpleImputer(strategy='mean')
Xtr = imputer.fit_transform(Xtr)
Xval = imputer.fit_transform(Xval)

**Scaling**

As redes neurais são bem senssíveis a valores das variável pois elas tem relação imediata com os pesos que são aprendidos pela rede. Assim para evitar que determainada feature tenha imfluência maior que outro por conta de deferença na escala dos valores, vamos aplicar a técnica de **scaling** para que todo os valores do dataset estejam dentro de um mesma escala de valores.

In [52]:
# aplicando o scaling dos valores
scaler = MaxAbsScaler()
Xtr = scaler.fit_transform(Xtr)
Xval = scaler.fit_transform(Xval)

In [56]:
Xtr

array([[0.10526316, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.31578947, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.47368421, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.47368421, 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.63157895, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.31578947, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
# ytr = ytr.values
# yval = yval.values

**Montando a Rede**

In [64]:
tf.random.set_seed(2)

inp = tfk.layers.Input((Xtr.shape[1], )) # camada de input definindo o tamanho
hid1 = tfk.layers.Dense(100, activation='relu')(inp)
drop = tfk.layers.Dropout(0.5)(hid1)
hid2 = tfk.layers.Dense(18, activation='relu')(drop)
out = tfk.layers.Dense(1, activation='relu')(hid2)

mdl = tfk.Model(inp, out)
mdl.compile(loss='msle', optimizer='adam')
mdl.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 285)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               28600     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 18)                1818      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 19        
Total params: 30,437
Trainable params: 30,437
Non-trainable params: 0
_________________________________________________________________


Score aceitavel para a cometição de origem dos dados = 0.014

In [65]:
np.sqrt(144)

12.0

**Early Stopping**

Uma questão importante para p treinamento de uma rede neural é o EarlyStoping. Essa cofiguração indica para a rede quando ela deve, denro de um epoc, parar de tentar encontrar um loss mais baixo em sua estratégia de Gradinte Descendente.

Vamos cofigurar o early stop para acompanhar a evolução da métrica val_loss (monitor='val_loss') e, se em 10 tentativas (patience=10) a mética não melhorar pelo menos em 10% (min_delta=0.1), então os pesos são guardados e se incia uma nova epoch.

In [66]:
es = tfk.callbacks.EarlyStopping(
    monitor='val_loss', 
    min_delta=0.1,
    patience=10,
    mode='min', 
    restore_best_weights=True
)

**Treinando o modelo**

In [75]:
mdl.fit(Xtr,
        ytr,
        validation_data=(Xval,yval),
        epochs=1000,
        shuffle=True,
        batch_size=1,
        callbacks=[es])

Epoch 1/1000
730/730 [==============================] - 1s 2ms/step - loss: 0.1374 - val_loss: 0.0904
Epoch 2/1000
730/730 [==============================] - 1s 2ms/step - loss: 0.1168 - val_loss: 0.0820
Epoch 3/1000
730/730 [==============================] - 1s 2ms/step - loss: 0.1147 - val_loss: 0.0728
Epoch 4/1000
730/730 [==============================] - 1s 2ms/step - loss: 0.0956 - val_loss: 0.0660
Epoch 5/1000
730/730 [==============================] - 1s 2ms/step - loss: 0.0921 - val_loss: 0.0611
Epoch 6/1000
730/730 [==============================] - 1s 2ms/step - loss: 0.0899 - val_loss: 0.0542
Epoch 7/1000
730/730 [==============================] - 1s 2ms/step - loss: 0.0758 - val_loss: 0.0503
Epoch 8/1000
730/730 [==============================] - 1s 2ms/step - loss: 0.0705 - val_loss: 0.0470
Epoch 9/1000
730/730 [==============================] - 1s 2ms/step - loss: 0.0690 - val_loss: 0.0443
Epoch 10/1000
730/730 [==============================] - 1s 2ms/step - loss: 0.069

**Fazendo Predições**

Agora que temos o modelo treinado podemos fazer a predição do dataset de validação yval e calcular o MSLE na comparação dos valores.

In [79]:
ypred = mdl.predict(Xval)
from sklearn.metrics import  mean_squared_log_error
np.sqrt(mean_squared_log_error(yval,ypred))
# epocks = 10 ==> msle=0.605055516476766

0.3005847522926521